## Changing parameters in a Mimi model

This is meant as a simple hands-on example that can be readily executed and adapted. More thorough documentation, explanation and examples can be found [here](https://www.mimiframework.org/Mimi.jl/stable/tutorials/tutorial_3/).

In [ ]:
using Pkg;
Pkg.activate(".") # activate the environment in the "modelling" folder
Pkg.instantiate() # install the packages in the environment

### 1. Modifying DICE parameters

Let's start by changing something in the DICE model. Here's a table with parameters that might be worthwile playing around with:

| Parameter symbol | Component | Name and meaning | Source code |
| --- | --- | --- | --- | 
| `:a2` | `:damages` | quadratic coefficient of damage function| [&rarr;](https://github.com/felixschaumann/OptMimiDICE2016R2.jl/blob/main/src/components/damages_component.jl) |
| `:t2xco2` | `:climatedynamics` | equilibrium climate sensitivity | [&rarr;](https://github.com/felixschaumann/OptMimiDICE2016R2.jl/blob/main/src/components/climatedynamics_component.jl) |
| `:elasmu` | `:welfare` | elasticity of marginal utility of consumption | [&rarr;](https://github.com/felixschaumann/OptMimiDICE2016R2.jl/blob/main/src/components/welfare_component.jl) |
| `:rr` | `:welfare` | social discount rate (factor?) | [&rarr;](https://github.com/felixschaumann/OptMimiDICE2016R2.jl/blob/main/src/components/welfare_component.jl) |

Note that when you adapt a certain parameter and run the model again, all the model variables will update according to the new equations and parameter values, but the model will **not optimise welfare**! `MimiDICE2016R2` is a simulation model that does not feature optimisation. That is, it can be used to evaluate the SCC along a certain path, but it does not welfare-optimise. If you want to be able to optimise, check out the [`OptMimiDICE2016R2` package](https://github.com/felixschaumann/OptMimiDICE2016R2.jl).

In [1]:
using Mimi
using MimiDICE2016R2

In [2]:
m_DICE = MimiDICE2016R2.get_model()
run(m_DICE)

`m_DICE` is a standard DICE model for which we can explore results with `explore(m_DICE)`. Before we do that, let's change some parameters.

In [3]:
# Howard & Sterner damage specification including productivity effect
update_param!(m_DICE, :damages, :a2, 0.01145)
run(m_DICE)

m_DICE[:damages, :DAMAGES]

100-element Vector{Union{Missing, Float64}}:
    0.8700933704697317
    1.4810364421566184
    2.3878518383878085
    3.6833113466788383
    5.475613749038234
    7.888074585031499
   11.05818951276641
   15.13608488705448
   20.282408210156422
   26.66574135472514
    ⋮
 3937.0649850081945
 3991.3852369663946
 4040.791758544711
 4086.4900549479926
 4129.334696670338
 4169.9332779923225
 4208.718869189598
 4246.000884959788
 4282.000982461122

The last line, following the syntax `model[:component, :variable]`, let's you look at the vector of time steps of a certain variable. In this case, we can see, how the damages over time change after having altered a damage function parameter.

In [4]:
# A particularly low climate sensitivity (normally around 3)
update_param!(m_DICE, :climatedynamics, :t2xco2, 2)

m_DICE[:climatedynamics, :TATM]

100-element Vector{Union{Missing, Float64}}:
 0.85
 1.0163451840920545
 1.189205773623509
 1.3678107128004386
 1.551456319943855
 1.7394509386676342
 1.931092168386334
 2.1256628965081843
 2.322437285797461
 2.520691184073483
 ⋮
 6.6072488696504825
 6.597907890227782
 6.588642708861447
 6.5794515944405
 6.570333459950364
 6.561287497939146
 6.552312978419538
 6.543409143666289
 6.534575159624617

In [16]:
# A particular high climate sensitivity of 4
update_param!(m_DICE, :climatedynamics, :t2xco2, 4)

m_DICE[:climatedynamics, :TATM]

100-element Vector{Union{Missing, Float64}}:
 0.85
 0.9605512231646353
 1.0775134673170186
 1.19970203223599
 1.3261359352238185
 1.4559510648550629
 1.5883558451183455
 1.722611473610401
 1.8580254784101915
 1.9939517230297052
 ⋮
 4.3204810385957515
 4.313383047036012
 4.306368017280178
 4.299433256521963
 4.2925769738068205
 4.285797773028904
 4.279094381726255
 4.272465516846562
 4.265909827812359

Let's calculate the SCC. See [here](https://github.com/anthofflab/MimiDICE2016R2.jl?tab=readme-ov-file#calculating-the-social-cost-of-carbon).

In [17]:
# Get the social cost of carbon in year 2020 from the default MimiDICE2016R2 model:
# Pure Rate of time Preference is 0.03, meaning that the utility in the future is dicounted by 3%
scc = MimiDICE2016R2.compute_scc(m_DICE, year=2020, prtp=0.03)

120.37126881644173

Now you can experiment which SCC values you get based on different parameters for the damage function, climate sensitivity, marginal elasticity of consumption, or the rate of pure time preference (`prtp` in the `compute_scc` function).

In [18]:
# Get the social cost of carbon in year 2100 from the default MimiDICE2016R2 model:
scc = MimiDICE2016R2.compute_scc(m_DICE, year=2100, prtp=0.03)

1366.4226366407513

In [19]:
# Get the social cost of carbon with a pure rate of time preference of 0 (meaning that the utility of the future is valued just as much as the utility in the present) from the default MimiDICE2016R2 model in the year 20100:
scc = MimiDICE2016R2.compute_scc(m_DICE, year=2100, prtp=0.00)

85048.57118330259

One can conclude: The higher the Climate Sensitivity and the lower the PRTP the higher the SCC. That makes absolutly sense.

Let's change another paramter: The Elasticity of marginal utility of consumption

In [25]:
# The elasticity of marginal utility of consumption increases up to 1.7.
update_param!(m_DICE, :welfare, :elasmu, 1.7)

m_DICE[:welfare, :PERIODU]

100-element Vector{Union{Missing, Float64}}:
 0.44863518477148
 0.48789141031006333
 0.5236441549145197
 0.5561796819149716
 0.5857690150435622
 0.6126611921831264
 0.637081656759988
 0.6592329791698304
 0.6792965052073883
 0.6974342563189733
 ⋮
 0.8696437783108346
 0.8741081196543277
 0.8782793899988677
 0.8822208021155236
 0.8859761660958327
 0.8895762836885224
 0.8930431402641932
 0.8963926955304269
 0.8996367622972194

In [24]:
#Originaly the elasticity of marginal utility of consumption is at 1.45.
#Problem: NOTHING CHANGES: What is wrong?
update_param!(m_DICE, :welfare, :elasmu, 1.45)

m_DICE[:welfare, :PERIODU]

100-element Vector{Union{Missing, Float64}}:
 0.44863518477148
 0.48789141031006333
 0.5236441549145197
 0.5561796819149716
 0.5857690150435622
 0.6126611921831264
 0.637081656759988
 0.6592329791698304
 0.6792965052073883
 0.6974342563189733
 ⋮
 0.8696437783108346
 0.8741081196543277
 0.8782793899988677
 0.8822208021155236
 0.8859761660958327
 0.8895762836885224
 0.8930431402641932
 0.8963926955304269
 0.8996367622972194

### 2. Modifying META parameters

Let's first go into the directory of the META-2021 model.

In [ ]:
cd("./META-2021/src") # change directory
include("../src/MimiMETA.jl")

In [ ]:
include("./META-2021/src/MimiMETA.jl")

Sometimes, it is necessary to run a simple `model` first. Seems to be a bug, but don't worry about ut.

In [ ]:
model = base_model();
run(model);

In [ ]:
m_base = base_model(rcp="RCP4.5", ssp="SSP2", tdamage="pointestimate", slrdamage="mode");
run(m_base);

m_full = full_model(rcp="RCP4.5", ssp="SSP2", tdamage="pointestimate", slrdamage="mode");
run(m_full);

In [ ]:
include("./META-2021/scc.jl") # include the scc calculation file

scc_base = calculate_scc(m_base, 2020, 10., 1.5) # year 2020, emissions pulse 10Gt CO2, elasticity of marginal utility 1.5
scc_full = calculate_scc(m_full, 2020, 10., 1.5)

Again, as in the case for DICE, you can try to vary parameters, look at the current parameter values with `model[:component, :parameter]`, and change them using `update_param!` and `run`. 

See more information on running the META model and calculating SCC values [here](https://github.com/openmodels/META-2021?tab=readme-ov-file#mimi-model).